# Microsoft Dataverse Investigations Notebook

---

## Pre-requisites
- Log Analytics *Reader* permissions on the Microsoft Sentinel workspace
- Read access to a target Microsoft Dataverse instance
- Virus Total or alternative MSTICPy TI providers
- Maxmind GeoLite2 provider
- Python 3.8 notebook kernel

# Contents

1. **Introduction**<br>
2. **Initializing the Microsoft Dataverse query provider**<br>
   2.1 Initialize Notebook and connect to Dataverse<br>
   2.2 Connect to Microsoft Sentinel<br>
   2.3 List available queries<br><br>
3. **Search for a record across entities**<br>
   3.1 (Optional) Example query to acquire record details<br>
   3.1 Enter a record id (GUID) value<br>
   3.2 Search common entites<br><br>
4. **Check suspicious files uploaded to Dataverse**<br>
   4.1 Set query time boundaries<br>
   4.2 Testing VirusTotal Lookup<br><br>
5. **Identify custom tables without auditing enabled**<br>
   5.1 List publishers<br>
   5.2 Display custom tables with audit status<br><br>
6. **Direct query on Dataverse audit table**<br>
   6.1 Enter user name (UPN) to search<br>
   6.2 Set query time boundaries<br>
   6.3 Visualize event timeline from a direct query on Dataverse audit table<br><br>
7. **Investigate suspicious IP addresses**<br>
   7.1 Extract IP addresses from Sentinel alert<br>
   7.2 Visualize IP Geolocation<br>
   7.3 Lookup IP addresses in TI<br><br>
8. **Conclusion and Next Steps**<br>
9. **Further Resources**<br>


---

# 1. Introduction

This notebook provides threat hunting and security utility features by leveraging the Dataverse Web API. 

<div style="color: Black; background-color: LightGoldenRodYellow; padding: 5px; font-size: 20px">
<p>Warning. Due to rendering issues in Azure Machine Learning, we strongly recommend running this notebook in <b>Jupyter Lab</b> or <b>VSCode</b>.</p>
<div style="font-size: 14px">
To do this:
<ul>
<li>Click on the notebook toolbar menu - the &#8803; symbol in top left of the notebook</li>
<li>Select the <b>Editors</b> option and choose either <b>JupyterLab</b> or <b>VSCode</b></li>
<li>When prompted to stay in AML click the <b>Continue</b> button.
<li>The notebook should open in another browser tab</li>
</ul>
The MSTICPy settings editor uses notebook widgets, which are not
fully supported in AML notebooks.
</div>
</div>


---

# 2. Initializing the Microsoft Dataverse query provider

To do...

---

## 2.1 Initialize Notebook and connect to Dataverse

MSTICPy includes support for Dataverse via an OData driver, utilizing a standard model for use in Sentinel notebooks. The OData driver will connect to a pre-defined Dataverse instance configued in MSTICPyconfig.yaml or during runtime.

In [4]:
import msticpy as mp
from msticpy.context.geoip import GeoLiteLookup, IPStackLookup
from msticpy.data.drivers.dataverse_driver import DataverseDriver
from msticpy.datamodel.entities import File, IpAddress
from msticpy.nbtools import WorkspaceConfig
from msticpy.nbwidgets import Progress, GetText, OptionButtons, QueryTime, SelectSubset
from msticpy.vis.data_viewer import DataViewer
from msticpy.nbtools import *
from msticpy.sectools import *

import pandas as pd

mp.init_notebook()
qry_dataverse = mp.QueryProvider("Dataverse")
qry_dataverse.connect(
    username="wiking@sentinelbizapps.com",
)


Connected.


---

## 2.2 Connect to Microsoft Sentinel

The **cell** below is a code cell (note that it looks different from the
cell you are reading). The current cell is known as a *Markdown* cell
and lets you write text (including HTML) and include static images.

In [6]:
# Try to read workspace configuration from MSTICPyconfig.yaml, and then authenticate
ws_config = WorkspaceConfig(workspace="Default")
la_qry_prov = mp.QueryProvider(data_environment="LogAnalytics")
la_qry_prov.connect(connection_str=ws_config.code_connect_str)


Connecting... connected


---

## 2.2 List available queries

A number of pre-defined queries are included as part of msticpy. Additional custom queries and community contributions can be added using the following guide: <a href>https://guide.com</a><br>
Use the following code snippet to list all queries available via the Dataverse driver...

In [7]:
qry_dataverse.list_queries()

['DataverseQuery.get_annotations_by_date',
 'DataverseQuery.get_audits',
 'DataverseQuery.get_record_by_recordid',
 'DataverseQuery.get_signins_by_user',
 'DataverseQuery.get_systemuser_by_id',
 'DataverseQuery.get_systemuser_by_upn',
 'DataverseQuery.list_accounts',
 'DataverseQuery.list_annotations',
 'DataverseQuery.list_entities',
 'DataverseQuery.list_publishers',
 'DataverseQuery.list_systemusers']

---

# 3. Search for a record across entities

During an incident, records modified often show up in the logs using record ID (GUID) and this makes deciphering exact activities difficult. For this scenario we'll use the Dataverse Query Provider to examine common entities, searching for a match and returning details about the record in question.

---

## 3.1 (Optional) Example query to acquire record details

In the code cell below, we have a KQL query that will return all ExportToExcel events for the given time period. In this example, we'll take a single row and display the QueryResults column. This column contains a list of record id's as they show up in the logs. Copy one of the record GUIDs to the clipboard and proceed.

In [8]:
sentinel_query = """Dynamics365Activity
| where TimeGenerated >=ago(190d)
| where Message == 'ExportToExcel'
| project TimeGenerated, UserId, ClientIP, Message, QueryResults
| take 1"""

sentinel_df = la_qry_prov.exec_query(query=sentinel_query)
print(sentinel_df.QueryResults.tolist())

['a4cea450-cb0c-ea11-a813-000d3a1b1223,da59721f-02b2-ea11-a812-000d3a1b14a2,dbdd0b93-4a1b-4848-b83a-39352f6b2e7a,a1488962-8788-4264-9ff0-c7c62258aecd']


---

## 3.2 Enter a record id (GUID) value

Run the code cell below and enter the record id GUID to search for. The value supplied will be used for our record search.

In [9]:
#"81af2bbb-7257-5b7b-8ee9-153ba1db21d2"
record_id = GetText(prompt='Enter a value', auto_display=True);


Text(value='', description='Enter a value', layout=Layout(width='50%'), style=DescriptionStyle(description_wid…

---

## 3.3 Search common entites



In [10]:
common_entities = [
    "accounts", "activitymimeattachments", "annotations", "bulkoperations", "businessunits", "campaigns",
    "competitors", "contacts", "contracts", "discounts", "discounttypes", "emails", 
    "entitlements", "expiredprocesses", "fieldpermissions", "fieldsecurityprofiles", "goalrollupqueryies", "goals",
    "incidents", "invoices", "kbarticles", "knowledgearticles", "leads", "leadtoopportunitysalesprocesses",
    "lists", "metrics", "newprocesses", "opportunities", "opportunitysalesprocesses", "organizations", 
    "phonetocaseprocesses", "positions", "pricelevels", "productpricelevels", "products", "queueitems",
    "quotes", "reports", "roles", "salesliteratures", "services", "sharepointsites", "socialprofiles", 
    "systemusers", "teams", "teamtemplates", "templates", "transactioncurrencies", "translationprocesses",
]

entities = qry_dataverse.DataverseQuery.list_entities()
matched_entities = entities[entities["EntitySetName"].isin(common_entities)]
progress = Progress(completed_len=len(common_entities))
counter = 0
for index, row in matched_entities.iterrows():
    entity_name = row["EntitySetName"]
    id_property = row["PrimaryIdAttribute"]
    try:
        query_result = qry_dataverse.DataverseQuery.get_record_by_recordid(
            entity_name=entity_name,
            id_property=id_property,
            record_id=record_id.value,
        )
        if not query_result.empty:
            matched_record, entity_name = query_result, entity_name
            break
        counter += 1
        progress.update_progress(new_total=counter)
    except:
        raise Exception("Error with query, please check if record id provided was a valid GUID")
progress.hide()
if not(type(matched_record) is property) and not(pd.DataFrame(matched_record).empty):
    print(f"Record was found in entity: {entity_name}")
    print(matched_record.transpose())
else:
    print("Record was not found in searched entities")


Exception: Error with query, please check if record id provided was a valid GUID

---

# 4. Check suspicious files uploaded to Dataverse

In this scenario, we'll examine files uploaded to Dataverse and use Virus Total to scan file hash against a database of known malicous files. The following 

<li>VirusTotal
<li>AlienVault OTX
<li>RiskIQ
<li>IBM XForce
<li>MS Sentinel TI
<li>GreyNoise

---

## 4.1 Set query time boundaries
   

In [11]:
q_times = QueryTime(units='day', max_before=20, before=5, max_after=1, auto_display=True)


---

## 4.2 Select files to analyze


In [12]:
annotations = qry_dataverse.DataverseQuery.get_annotations_by_date(
    start=q_times.start.isoformat(), end=q_times.end.isoformat()
)

if len(annotations.index) > 0:
    selection = {v: k for k, v in annotations["filename"].to_dict().items()}
    selected_files = SelectSubset(source_items=selection, auto_display=True)
else:
    print("No files matching selected query time boundaries")

---

## 4.3 Perform TI lookup

In [13]:
dec_df = base64.unpack_df(data=annotations.iloc[selected_files.selected_values], column='documentbody')
file_hashes = dec_df.sha256.to_list()
File.ti.lookup_file_hash(data=file_hashes)

Observables processed:   0%|          | 0/1 [00:00<?, ?obs/s]

,QuerySubtype,Result,Details,RawResult,Reference,Status,Ioc,IocType,SafeIoc,Severity,Provider
0,None,True,"{'verbose_msg': 'Scan finished, information embedded', 'response_code': 1, 'positives': 0, 'reso...","{'scans': {'Bkav': {'detected': False, 'version': '1.3.0.9899', 'result': None, 'update': '20221...",https://www.virustotal.com/vtapi/v2/file/report,0,972efbb0e7990a0b8404bbf9c7a57b047db169628aba7a017fd815ee5202e4d3,sha256_hash,972efbb0e7990a0b8404bbf9c7a57b047db169628aba7a017fd815ee5202e4d3,information,VirusTotal


---

# 5. Identify custom tables without auditing enabled

For effective monitoring and detection coverage, enablement of entity level audit settings is critical. Dataverse tables are not audit enabled by default and therefore need to be configured so that information such as changes to records , record additions and deletions and retrieval events are logged in Sentinel. Microsoft Sentinel Solution for Dynamics 365 provides a package to install common audit settings however, organizations will often create their own custom entities and these will need to audit enabled.

---

## 5.1 List publishers

In [15]:
publishers = qry_dataverse.DataverseQuery.list_publishers()
publishers

,@odata.etag,_organizationid_value,address1_addressid,uniquename,friendlyname,_createdby_value,publisherid,customizationoptionvalueprefix,customizationprefix,modifiedon,_modifiedby_value,versionnumber,isreadonly,createdon,address2_addressid,address2_line1,pinpointpublisherdefaultlocale,address1_county,address2_utcoffset,address2_fax,entityimage_url,address1_name,address1_line1,address1_postalcode,address2_line3,...,entityimage_timestamp,address1_latitude,address2_latitude,address1_longitude,address1_line2,supportingwebsiteurl,address2_line2,address2_postalcode,address2_upszone,address2_longitude,address1_fax,_createdonbehalfby_value,address2_country,description,address1_shippingmethodcode,address1_postofficebox,address1_upszone,address1_addresstypecode,address1_country,entityimageid,entityimage,address1_telephone3,address1_city,address1_telephone2,address1_telephone1
0,"W/""2686367""",0e5a49f4-cd8a-4579-a405-e26e92953595,b7170297-ffd1-4b5a-aab2-c07afe72f8a7,Cr36ce0,CDS Default Publisher,dad93ab1-d190-4691-adac-ba32798fc799,00000001-0000-0000-0000-00000000005a,79618,crf80,2022-12-19T16:35:00Z,dad93ab1-d190-4691-adac-ba32798fc799,2686367,False,2022-12-19T16:35:00Z,e3ab8ae8-c7f8-4705-b51d-9a6e6bb5c9e6,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,NaN,None,None,None,None,None,None,None
1,"W/""3059224""",0e5a49f4-cd8a-4579-a405-e26e92953595,None,MicrosoftCorporation,MicrosoftCorporation,dad93ab1-d190-4691-adac-ba32798fc799,d21aab70-79e7-11dd-8874-00188b01e34f,20000,,2023-01-09T09:43:22Z,dad93ab1-d190-4691-adac-ba32798fc799,3059224,False,2022-12-19T16:19:08Z,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,Default publisher for this organization,NaN,None,None,NaN,None,None,None,None,None,None,None
2,"W/""2827825""",0e5a49f4-cd8a-4579-a405-e26e92953595,7e608de9-0917-4c2f-8609-200b6254c6d3,DefaultPublishercontosocrm,Default Publisher for contoso-crm,dad93ab1-d190-4691-adac-ba32798fc799,d21aab71-79e7-11dd-8874-00188b01e34f,10000,new,2022-12-21T16:04:01Z,dad93ab1-d190-4691-adac-ba32798fc799,2827825,False,2022-12-19T16:19:08Z,0ecbe3ba-f07e-48be-b64f-fc557a896f68,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,Default publisher for this organization,NaN,None,None,NaN,None,None,None,None,None,None,None
3,"W/""2686370""",0e5a49f4-cd8a-4579-a405-e26e92953595,None,microsoftfirstparty,Default Publisher for CITTest,dad93ab1-d190-4691-adac-ba32798fc799,d21aab72-79e7-11dd-8874-00188b01e34f,20000,,2022-12-19T16:35:00Z,dad93ab1-d190-4691-adac-ba32798fc799,2686370,False,2022-12-19T16:35:00Z,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,Default publisher for this organization,NaN,None,None,NaN,None,None,None,None,None,None,None
4,"W/""3095019""",0e5a49f4-cd8a-4579-a405-e26e92953595,643129f5-bfc7-4e56-9c19-ac3179daa3ab,microsoftdynamics,Dynamics 365,dad93ab1-d190-4691-adac-ba32798fc799,4c38ac96-0a29-4045-8443-1f412fcc30bc,19236,msdyn,2023-01-10T04:41:44Z,dad93ab1-d190-4691-adac-ba32798fc799,3095019,True,2022-12-19T16:51:45Z,fd003d53-db19-45b8-a42d-a0dcbf55f632,None,None,None,None,None,None,None,One Microsoft Way,98052,None,...,None,None,None,None,None,http://crm.dynamics.com,None,None,None,None,None,None,None,Dynamics 365,1.0,None,None,1.0,US,None,None,None,Redmond,None,1-800-Microsoft(642-7676)
5,"W/""2943739""",0e5a49f4-cd8a-4579-a405-e26e92953595,f07e6ab1-9efb-4100-b5ef-8c1fc0b18f27,dynamics365customerengagement,Dynamics 365,dad93ab1-d190-4691-adac-ba32798fc799,bff34d88-35e1-44e2-9f65-708e24566b28,10023,msdynce,2023-01-08T02:24:58Z,dad93ab1-d190-4691-adac-ba32798fc799,2943739,True,2022-12-19T17:55:57Z,94226c8a-1fa1-44c8-9748-a62d7d28fad8,None,None,None,None,None,None,None,One Microsoft Way,98052,None,...,None,None,None,None,None,http://cr

---

## 5.2 Display custom tables with audit status

Whenever an unmanaged customization is made to Dataverse, the change assumes the solution of the CDS Default Publisher. Publishers are are assigned a prefix, where any new entity is created with a solution publisher prefix before the table name. Using the publisher information gathered in the previous cell, we can identify custom tables via the solution prefix of the CDS Default Publisher of the target Dataverse instance.

In [16]:
publisher_id = '00000001-0000-0000-0000-00000000005a'
entities = qry_dataverse.DataverseQuery.list_entities()
default_publisher = publishers.loc[publishers['publisherid'] == publisher_id]
prefix = default_publisher.customizationprefix[0]
custom_entities = entities.loc[entities['EntitySetName'].str.startswith(prefix)]
custom_entities


,EntitySetName,SchemaName,PrimaryIdAttribute,IsRetrieveAuditEnabled,IsRetrieveMultipleAuditEnabled,MetadataId,IsAuditEnabled.Value,IsAuditEnabled.CanBeChanged,IsAuditEnabled.ManagedPropertyLogicalName
130,crf80_contosocustoms,crf80_contosocustom,crf80_contosocustomid,False,False,383675f4-fa7f-ed11-81ad-000d3a34389f,False,True,canmodifyauditsettings


---

# 6. Direct query on Dataverse audit table

---

## 6.1 Enter user name (UPN) to search

In [18]:
# Must be enclosed with single quotes for now
user_principal_name = GetText(prompt='Enter a UPN to search', auto_display=True)

Text(value="'wiking@sentinelbizapps.com'", description='Enter a UPN to search', layout=Layout(width='50%'), st…

---

## 6.2 Set query time boundaries


In [19]:
signin_window = QueryTime(units='day', max_before=20, before=5, max_after=1, auto_display=True)

In [ ]:
#systemuser = qry_dataverse.DataverseQuery.get_systemuser_by_upn(upn=user_principal_name.value)
#qry_dataverse.DataverseQuery.get_systemuser_by_id(systemuserid=systemuser.systemuserid[0])
#qry_dataverse.DataverseQuery.get_record_by_recordid(entity_name='systemusers',id_property='systemuserid', record_id='6deeb58e-2082-ed11-81ad-00224805fd33')

---

## 6.3 Visualize event timeline from a direct query on Dataverse audit table

In [20]:
test = qry_dataverse.DataverseQuery.get_signins_by_user(start=signin_window.start.isoformat(), end=signin_window.end.isoformat(), systemuserid='f80937d5-b87f-ed11-81ad-000d3a34389f')
from msticpy.vis.timeline import display_timeline
# At a minimum we need to pass a dataframe with timestamp column
# (defaults to TimeGenerated)
test.mp_plot.timeline(time_column='createdon', source_columns=['action'])

Loading BokehJS ...

Column(id='1075', ...)

---

# 7. Investigate suspicious IP addresses

---

## 7.1 Extract IP addresses from Sentinel alert

In [21]:
#mp.MpConfigEdit()
signin_query = """Dynamics365Activity
| where TimeGenerated >=ago(190d)
| where UserId == "labadmin@azwk.onmicrosoft.com"
| summarize by ClientIP"""
sentinel_df = la_qry_prov.exec_query(signin_query)

iplocation = GeoLiteLookup()
sentinel_df.mp_plot.folium_map(ip_column='ClientIP', )



---

## 7.2 Lookup IP addresses against TI

In [22]:
IpAddress.ti.lookup_ip(data=sentinel_df, input_column="ClientIP")

Observables processed:   0%|          | 0/25 [00:00<?, ?obs/s]

,QuerySubtype,Result,Details,RawResult,Reference,Status,Ioc,IocType,SafeIoc,Severity,Provider
0,None,True,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 0, 'detected_urls': []}","{'detected_urls': [], 'asn': 13037, 'country': 'GB', 'response_code': 1, 'as_owner': 'Zen Intern...",https://www.virustotal.com/vtapi/v2/ip-address/report,0,82.71.12.40,ipv4,82.71.12.40,information,VirusTotal
0,None,True,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 0, 'detected_urls': []}","{'asn': 8075, 'undetected_urls': [['http://20.73.127.158/', '243d07fc85ca5881f410451bbf4edcf8d21...",https://www.virustotal.com/vtapi/v2/ip-address/report,0,20.73.127.158,ipv4,20.73.127.158,information,VirusTotal
0,None,True,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 0, 'detected_urls': []}","{'detected_urls': [], 'asn': 8075, 'country': 'NL', 'response_code': 1, 'as_owner': 'MICROSOFT-C...",https://www.virustotal.com/vtapi/v2/ip-address/report,0,20.73.126.196,ipv4,20.73.126.196,information,VirusTotal
0,None,True,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 0, 'detected_urls': []}","{'detected_urls': [], 'asn': 8075, 'country': 'NL', 'response_code': 1, 'as_owner': 'MICROSOFT-C...",https://www.virustotal.com/vtapi/v2/ip-address/report,0,20.73.127.169,ipv4,20.73.127.169,information,VirusTotal
0,None,False,No Content,<Response [204 No Content]>,https://www.virustotal.com/vtapi/v2/ip-address/report,204,20.90.131.124,ipv4,20.90.131.124,information,VirusTotal
0,None,False,No Content,<Response [204 No Content]>,https://www.virustotal.com/vtapi/v2/ip-address/report,204,51.140.74.14,ipv4,51.140.74.14,information,VirusTotal
0,None,False,No Content,<Response [204 No Content]>,https://www.virustotal.com/vtapi/v2/ip-address/report,204,20.76.46.131,ipv4,20.76.46.131,information,VirusTotal
0,None,False,No Content,<Response [204 No Content]>,https://www.virustotal.com/vtapi/v2/ip-address/report,204,51.137.167.176,ipv4,51.137.167.176,information,VirusTotal
0,None,False,No Content,<Response [204 No Content]>,https://www.virustotal.com/vtapi/v2/ip-address/report,204,51.143.208.217,ipv4,51.143.208.217,information,VirusTotal
0,None,False,No Content,<Response [204 No Content]>,https://www.virustotal.com/vtapi/v2/ip-address/report,204,51.137.167.177,ipv4,51.137.167.177,information,VirusTotal
